In [1]:
import pickle
import nest_asyncio
from datasets import load_dataset



from llama_index.core import (
    KnowledgeGraphIndex,
    StorageContext,
    Document
)
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor

from llama_index.embeddings.huggingface import HuggingFaceEmbedding

from llama_index.llms.llama_cpp import LlamaCPP

/mloscratch/homes/ordonnea/conda/envs/graph_rag/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mloscratch/homes/ordonnea/conda/envs/graph_rag/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_url" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/mloscratch/homes/ordonnea/conda/envs/graph_rag/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_path" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/mloscratch/homes/ordonnea/conda/envs/graph_rag/lib/python3.12/site-packages/pydant

In [2]:
model_url = "https://huggingface.co/QuantFactory/Meta-Llama-3-8B-Instruct-GGUF/resolve/main/Meta-Llama-3-8B-Instruct.Q4_0.gguf"
llm_llama3 = LlamaCPP(
    model_url=model_url,
    model_path=None,
    temperature=0.1,
    max_new_tokens=512,
    context_window=3900,
    generate_kwargs={},
    model_kwargs={"n_gpu_layers": -1},
    verbose=True,
)

embed_model = HuggingFaceEmbedding(model_name = "dunzhang/stella_en_400M_v5", device = "cuda", trust_remote_code=True, embed_batch_size=20)

llama_model_loader: loaded meta data with 27 key-value pairs and 291 tensors from /tmp/llama_index/models/Meta-Llama-3-8B-Instruct.Q4_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Models
llama_model_loader: - kv   3:                         general.size_label str              = 8.0B
llama_model_loader: - kv   4:                            general.license str              = llama3
llama_model_loader: - kv   5:                               general.tags arr[str,6]       = ["facebook", "meta", "pytorch", "llam...
llama_model_loader: - kv   6:                          general.languages arr[str,1]       = ["en"]
llama_model

In [3]:
graph_store = SimpleGraphStore()

storage_context = StorageContext.from_defaults(graph_store=graph_store)

In [18]:
def load_wikitext(split="train"):
    # Load WikiText dataset
    dataset = load_dataset("Salesforce/wikitext", "wikitext-103-raw-v1")
    
    # Process each article
    documents = []
    current_doc = ""
    current_title = ""
    
    for item in dataset[split]:
        text = item['text']
        if text.startswith(" = ") and text.endswith(" = \n"):
            # New article title found
            if current_doc:
                documents.append(Document(text=current_doc, metadata={"title": current_title}))
            current_title = text
            current_doc = ""
        else:
           current_doc += text + "\n"
    
    # Add the last document
    if current_doc:
        documents.append(Document(text=current_doc, metadata={"title": current_title}))
    
    return documents

# Load WikiText documents
wikitext_docs = load_wikitext()
print(f"Loaded {len(wikitext_docs)} documents from WikiText")

Loaded 305075 documents from WikiText


In [23]:
print(wikitext_docs[3].metadata)

{'title': ''}
